<a id="datagrids-top"></a>
<div style="width: 50px; height: 50px; border-radius: 50%; background: #F1C40F; margin-bottom: 0px; text-align: center;">
    <img src="../img/Welcome Page/launch.png" style="max-height: 95%; max-width: 95%;">
</div>
<h1 style="margin-top: 20px; margin-bottom: 5px;">DataGrids<br><span style="color: orange; margin-bottom: 0px;">Display Data in an Interactive Table</span></h1>
<h4 style="color: orange; margin-top: 0px;">(20-min Read)</h4>

By the end of this document, you will be able to:
1. Render multiple objects to a Jupyter cell's output using `IPython.display.display()`
1. Initialize and populate an interactive DataGrid
1. Apply `TextRenderer` objects to customize the appearance of a DataGrid

### Contents
- [The BQuant Project Flow](#dg-flow)
- [The IPython Display Function](#dg-ipy)
- [Initialize a DataGrid](#dg-init)
- [Basic DataGrid Customization](#dg-cust)
- [Additional Resources](#dg-help)

<a href="1 BQL Concepts.ipynb">&larr; Back to BQL Concepts</a>&emsp; | &emsp;
<a href="3 Charting.ipynb">Continue to Launch: Charting &rarr;</a>

---
<a id="dg-flow"></a>
<h1>BQuant Project Flow<br><span style="color: orange;">5-Step Process for Developing Your Projects</span></h1>

The 5-step project flow for BQuant development was mentioned in the Intro to BQuant section, but it bears repeating here. Every project you develop in BQuant will follow the same basic process below.

1. Setup Environment
1. Get Data from Bloomberg using BQL
1. Additional Data Munging (if necessary)
1. User Interface & Visualizations
1. Simplify, Document, & Publish

In this notebook, we cover the use of DataGrids in Step 4.

[&uarr; Return to Top](#datagrids-top)

---
<a id="dg-ipy"></a>
<h1><code style="background-color: transparent;">IPython.display</code><br><span style="color: orange;">Display Multiple Outputs From a Jupyter Cell</span></h1>

Now that you have some experience in Jupyter, you may have noticed that only the last line of code will make it to a cell's output. In the code snippet below, only the value of `y`, the last called variable, is displayed in output.

In [1]:
# Run this cell with the Run button in the Jupyter toolbar
x = 'This is X'
y = 'This is Y'

x
y

'This is Y'

For simple code like this, we can show both values by wrapping each variable in a `print()` statement, but this won't be sufficient for many non-string objects we wish to display. As the User Interface (UI) component of your apps becomes more complex, you'll eventually need to display two or more objects from one Jupyter cell: a DataGrid and a chart, for example.

One way we can show two or more items in a Jupyter cell's output is by leveraging the `IPython.display` library. This library has a function named `display()` which will tell Jupyter to include its argument in output. We can rewrite the code example above to display both variables.

In [2]:
# Run this cell with the Run button in the Jupyter toolbar
from IPython.display import display

x = 'This is X'
y = 'This is Y'

display(x)
display(y)

'This is X'

'This is Y'

Use the `display()` function when you need multiple UI elements to appear in a Jupyter cell's output.

[&uarr; Return to Top](#datagrids-top)

---
<a id="dg-init"></a>
<h1>Initialize a DataGrid<br><span style="color: orange;">Populate and Display an Interactive Table</span></h1>

DataGrids are UI elements that allow you to display data in an interactive table. Once displayed, the user can sort, filter, select, and even change cell values if the developer gives permission. Bloomberg has developed a Python library called `ipydatagrid` designed specifically for use in BQuant. Simply `import ipydatagrid` and pass a DataFrame to the `DataGrid` object to render your first grid.

In the example below, we use BQL to calculate average P/E ratios by country across members of the MSCI ACWI Index. This exercise is from the final "Try It Out" section of the previous <a href="./1 BQL Concepts.ipynb#dg-link-here">Launch: BQL Concepts</a> training. Run the cell below, and explore the resulting DataGrid. You can adjust column widths & row heights as well as sort & filter each column.

In [1]:
# Run this cell with the Run button in the Jupyter toolbar
# Step 1: Setup Environment
import bql
from ipydatagrid import DataGrid
bq = bql.Service()

# Step 2: Get Data From BQL
universe = bq.univ.members('BWORLD Index')
country = bq.data.country_full_name()
pe = bq.data.pe_ratio(fill='prev')
cntry_avg_pe = pe.group(country).avg()
sort_results = cntry_avg_pe.group().sort(order='desc')

req = bql.Request(universe, {'Average P/E': sort_results})
res = bq.execute(req)
df = res[0].df().head()

# Step 3: Additional Data Munging
df = df.set_index(str(country))        # <-- set the DataFrame's index to the country symbol
df.index.name = 'Country'              # <-- give the index a more readable name
df = df[['Average P/E']]               # <-- slice the DataFrame and keep only the Average P/E column

# Step 4: User Interface & Visualization
grid = DataGrid(df)
display(grid)

DataGrid(default_renderer=TextRenderer(), header_renderer=None)

<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice using a DataGrid after a BQL query.
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Write a BQL query to retrieve the last 10 years of Austria's nominal GDP (<code>PX_LAST</code> for <code>'ENGCAT Index'</code>). Use <code>per='Y'</code> to get year-end numbers. Remember to use the BQL Editor if you need help writing the query.</li>
            <li>Declare a variable named <code>austria_gdp</code> to hold the DataFrame resulting from your BQL request.</li>
            <li>Set the index on this DataFrame to the <code>'DATE'</code> column by calling <code>austria_gdp.set_index('DATE', inplace=True)</code></li>
            <li>Declare a variable named <code>my_grid</code> and use it to build a DataGrid for your BQL response.</li>
            <li>Display the DataGrid to the screen with <code>display(my_grid)</code></li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# import bql
# from ipydatagrid import DataGrid
# from IPython.display import display
# bq = bql.Service()

date_range = bq.func.range('-20Y', '0D')
data = bq.data.px_last(dates=date_range, per='Y')
request = bql.Request('ENGCAT Index', {'Austria GDP': data})
response = bq.execute(request)
austria_gdp = response[0].df()

austria_gdp.set_index('DATE', inplace=True)
my_grid = DataGrid(austria_gdp)
display(my_grid)

[&uarr; Return to Top](#datagrids-top)

---
<a id="dg-cust"></a>
<h1>Basic DataGrid Customization<br><span style="color: orange;">Control the Look and Style of your DataGrids</span></h1>

The `grid` we created in the last section is useful, but there is more we can do to improve the user experience. Running the cell above yields an output like this.

<img src="../img/Launch/grid_cust.png" style="height: 400px;">

<p>Let's fix the issues highlighted above. The following attributes of the `ipydatagrid.DataGrid` object will help.</p>
<ul>
    <li><code style="color: red; background-color: transparent;">layout</code> - pass a dictionary of CSS items to customize the look of the DataGrid figure. To limit the whitespace, we can reduce the height to 200 pixels.</li>
    <li><code style="color: orange; background-color: transparent;">renderers</code> - <code>ipydatagrid</code> has a <code>TextRenderer</code> object we can use to customize the number format displayed. We can use this same object to center align the numbers. We'll also use a second <code>TextRenderer</code> object to left align the country name.</li>
    <li><code style="color: yellow; background-color: transparent;">column_widths</code> - pass a dictionary of column names as keys and pixel integers as values to control the width of individual columns.</li>
</ul>

In [4]:
# Run this cell with the Run button in the Jupyter toolbar
from ipydatagrid import DataGrid, TextRenderer

# initialize TextRenderers for use in formatting grid
format_values = TextRenderer(format='.2f', horizontal_alignment='center')  # <-- use '.2f' to format decimals to 2 places
format_countries = TextRenderer(horizontal_alignment='left')

# build new DataGrid
new_grid = DataGrid(dataframe=df, 
                    layout={'height': '200px'},
                    renderers={'Country': format_countries,
                               'Average P/E': format_values},
                    column_widths={'Country': 100,
                                   'Average P/E': 100})
# display grid
display(new_grid)

DataGrid(column_widths={'Country': 100, 'Average P/E': 100}, default_renderer=TextRenderer(), header_renderer=…

<div style="border: 1px dotted orange; padding-left: 10px; padding-bottom: 10px;">
    <h3>&#9998; Try it out</h3>
    <p>
        Practice basic DataGrid Customization
        <ol>
            <li>Add a new Jupyter code cell below this one</li>
            <li>Declare a variable named <code>my_new_grid</code> and set it equal to a <code>DataGrid</code> object</li>
            <li>The DataFrame for this new grid should be the <code>austria_gdp</code> DataFrame you declared in the last exercise.</li>
            <li>Widen each of the columns on this DataGrid to 150 pixels.</li>
            <li>Format the <code>'DATE'</code> column to be more readable by using a date formatting TextRenderer such as <code>TextRenderer(format='%Y-%m-%d', format_type='time')</code></li>
            <li>Limit the whitespace around the grid by setting the <code>'height'</code> in <code>layout</code> to <code>'200px'</code>.</li>
            <li>Display the DataGrid to the screen with <code>display(my_new_grid)</code></li>
        </ol>
    We've written one possible solution to this problem in a hidden Python cell below. When you're ready to get a peek at an answer, click the <img src="../img/Controls/hidden_cell.png"> below.
    </p>
</div>

<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# from ipydatagrid import DataGrid, TextRenderer
# from IPython.display import display

my_new_grid = DataGrid(
    dataframe=austria_gdp,
    base_column_size=150,         # <-- widen all columns except index
    column_widths={'DATE': 150},  # <-- widen the index column
    layout={'height': '200px'},
    renderers={'DATE': TextRenderer(format='%Y-%m-%d', format_type='time')}
)

display(my_new_grid)


[&uarr; Return to Top](#datagrids-top)

---
<a id="dg-help"></a>
<h1>Additional Resources<br><span style="color: orange;">Find Out More About <code style="color: orange; background-color: transparent;">ipydatagrid</code></span></h1>

The `ipydatagrid` library has an extensive suite of customization options that are outside the scope of this tutorial. Check <a href="../exampleroot/341a99ff336b400695ccdd2b7212917d/0 Intro to ipydatagrid.ipynb">Example Projects &rarr; Visualization &rarr; ipydatagrid Tutorial & Migration Guide</a> to learn more about the tools provided in the library.

[&uarr; Return to Top](#datagrids-top)

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="1 BQL Concepts.ipynb">&larr; Back to BQL Concepts</a>&emsp;&emsp;
    <a href="#datagrids-top">&uarr; Return to Top</a>&emsp;&emsp;
    <a href="3 Charting.ipynb">Continue to Launch: Charting &rarr;</a>
    <br>
    <br>
    <a href="../Welcome.ipynb#welcome-top" style="font-size: 12px;">Return to the Welcome Page</a>
</p>